In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
0,229,SI,VIGENTE,TRÁNSITO RESTRINGIDO,26/01/2023,26/01/2023 15:30 HORAS,MADRE DE DIOS/TAHUAMANU/IBERIA,KM 599,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-11.387603,...,-11.387603+-69.492197,2023 15:30 HORAS+01+26+15:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
1,229,SI,VIGENTE,TRÁNSITO RESTRINGIDO,14/01/2023,26/01/2023 15:30 HORAS,MADRE DE DIOS/TAHUAMANU/TAHUAMANU,KM 549,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-11.665167,...,-11.665167+-69.229412,2023 15:30 HORAS+01+26+15:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
2,243,SI,VIGENTE,TRÁNSITO RESTRINGIDO,06/01/2023,26/01/2023 15:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 15:30 HORAS+01+26+15:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
3,242,SI,VIGENTE,TRÁNSITO RESTRINGIDO,14/01/2023,26/01/2023 15:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 489,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.1334806,...,-12.1334806+-69.1199462,2023 15:30 HORAS+01+26+15:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
4,240,SI,VIGENTE,TRÁNSITO RESTRINGIDO,16/01/2023,26/01/2023 15:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 469+320,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.2966504,...,-12.2966504+-69.1674238,2023 15:30 HORAS+01+26+15:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,343,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,26/01/2023 15:30 HORAS,UCAYALI/PADRE ABAD/IRAZOLA,KM 371,CENTRO POBLADO SAN JUAN BAUTISTA PARO INDEFINI...,-8.903362,...,-8.903362+-75.343754,2023 15:30 HORAS+01+26+15:30,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
98,343,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,26/01/2023 15:30 HORAS,UCAYALI/PADRE ABAD/IRAZOLA,KM 381,CENTRO POBLADO SAN PEDRO PARO INDEFINIDO SOLIC...,-8.945708,...,-8.945708+-75.388456,2023 15:30 HORAS+01+26+15:30,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
99,343,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,26/01/2023 15:30 HORAS,UCAYALI/PADRE ABAD/IRAZOLA,KM 385,CENTRO POBLADO HUIPOCA PARO INDEFINIDO SOLICIT...,-8.977655685,...,-8.977655685+-75.4442294,2023 15:30 HORAS+01+26+15:30,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
100,343,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,26/01/2023 15:30 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 408,CENTRO POBLADO BARRIO UNIDO PARO INDEFINIDO SO...,-9.03648944,...,-9.03648944+-75.49880659,2023 15:30 HORAS+01+26+15:30,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
